# Import the Observatory top shared csv and prepare it to be used in cross validation notebook

In [1]:
import gc

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
attributes_1 = pd.read_csv("_Spambee_Attributes__2022-04.csv")
attributes_2 = pd.read_csv("_Spambee_Attributes__2022-05.csv")
events_1 = pd.read_csv("_Spambee_Events__2022-04.csv")
events_2 = pd.read_csv("_Spambee_Events__2022-05.csv")
print(attributes_1.shape)
print(attributes_2.shape)
print(events_1.shape)
print(events_2.shape)

(220418, 5)
(237924, 5)
(19698, 5)
(19625, 5)


In [3]:
attributes = pd.concat([attributes_1, attributes_2], ignore_index=True)
events = pd.concat([events_1, events_2], ignore_index=True)

del attributes_1
del attributes_2
del events_1
del events_2

print(attributes.shape)
print(events.shape)

(458342, 5)
(39323, 5)


In [4]:
events["header"] = ''
events["from"] = ''
events["feedback_time"] = ''
events["host"] = ''
events.head()


,index,id,date,attribute_count,Tag,header,from,feedback_time,host
0,19625,287833,2022-04-01,28,None,,,,
1,19626,287834,2022-04-01,79,None,,,,
2,19627,287835,2022-04-01,89,Phishing,,,,
3,19628,287836,2022-04-01,159,Phishing,,,,
4,19629,287837,2022-04-01,62,None,,,,


## Merge the two different table to mix attributes with events joined on event_id

In [5]:
i=0
for event_id in events["id"]:
        event_attributes = attributes.loc[attributes["event_id"] == event_id]
        headers = event_attributes.loc[event_attributes["object_relation"] == "header", "value"]
        from_attribute = event_attributes.loc[event_attributes["object_relation"] == "from", "value"]
        feedback_time = event_attributes.loc[event_attributes["object_relation"] == "feedback_time", "value"]
        host = event_attributes.loc[event_attributes["object_relation"] == "host", "value"]
        events["header"][i] = ''.join(headers)
        events["from"][i] = '|'.join(from_attribute)
        events["feedback_time"][i] = '|'.join(feedback_time)
        events["host"][i] = '|'.join(host)
        i += 1

events.head()

C:\Users\Samuel\AppData\Local\Temp\ipykernel_20196\3642184332.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events["header"][i] = ''.join(headers)
C:\Users\Samuel\AppData\Local\Temp\ipykernel_20196\3642184332.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events["from"][i] = '|'.join(from_attribute)
C:\Users\Samuel\AppData\Local\Temp\ipykernel_20196\3642184332.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events["feed

,index,id,date,attribute_count,Tag,header,from,feedback_time,host
0,19625,287833,2022-04-01,28,None,Delivered-To: UNDISCLOSEDFORPRIVACY@PRIVACYDOM...,earleenlpwoodman2593@gmail.com|earleenlpwoodma...,2022-03-31T23:21:12.000000+0000|2022-03-31T23:...,www1.nyc.gov|www1.nyc.gov
1,19626,287834,2022-04-01,79,None,Delivered-To: UNDISCLOSEDFORPRIVACY@PRIVACYDOM...,quickbooks@notification.intuit.com|quickbooks@...,2022-04-01T00:26:28.000000+0000|2022-04-01T00:...,links.notification.intuit.com|links.notificati...
2,19627,287835,2022-04-01,89,Phishing,Received: from 10.194.205.88\r\n by atlas109.s...,fortune@msg.fortune.com|fortune@msg.fortune.com,2022-04-01T00:58:47.000000+0000|2022-04-01T00:...,fortune.com|fortune.com|fortune.com|fortune.co...
3,19628,287836,2022-04-01,159,Phishing,Received: from 10.213.249.35\r\n by atlas106.s...,miltongrovesc.yahoo.com@send.mailchimpapp.com|...,2022-04-01T00:55:55.000000+0000|2022-04-01T00:...,mailchi.mp|miltongrovesc.us15.list-manage.com|...
4,19629,287837,2022-04-01,62,None,Received: from 10.194.205.88\r\n by atlas103.s...,pedidos@riooffsite.com.br|pedidos@riooffsite.c...,2022-04-01T00:56:38.000000+0000|2022-04-01T00:...,u10225017.ct.sendgrid.net|usaa.com|u10225017.c...


In [6]:
events.to_csv("events.csv")

In [7]:
#replace tag by numerical value (0 for none 1 for phishing)
events.loc[events["Tag"] == "None", "Tag"] = 0
events.loc[events["Tag"] == "Phishing", "Tag"] = 1
events.head()

,index,id,date,attribute_count,Tag,header,from,feedback_time,host
0,19625,287833,2022-04-01,28,0,Delivered-To: UNDISCLOSEDFORPRIVACY@PRIVACYDOM...,earleenlpwoodman2593@gmail.com|earleenlpwoodma...,2022-03-31T23:21:12.000000+0000|2022-03-31T23:...,www1.nyc.gov|www1.nyc.gov
1,19626,287834,2022-04-01,79,0,Delivered-To: UNDISCLOSEDFORPRIVACY@PRIVACYDOM...,quickbooks@notification.intuit.com|quickbooks@...,2022-04-01T00:26:28.000000+0000|2022-04-01T00:...,links.notification.intuit.com|links.notificati...
2,19627,287835,2022-04-01,89,1,Received: from 10.194.205.88\r\n by atlas109.s...,fortune@msg.fortune.com|fortune@msg.fortune.com,2022-04-01T00:58:47.000000+0000|2022-04-01T00:...,fortune.com|fortune.com|fortune.com|fortune.co...
3,19628,287836,2022-04-01,159,1,Received: from 10.213.249.35\r\n by atlas106.s...,miltongrovesc.yahoo.com@send.mailchimpapp.com|...,2022-04-01T00:55:55.000000+0000|2022-04-01T00:...,mailchi.mp|miltongrovesc.us15.list-manage.com|...
4,19629,287837,2022-04-01,62,0,Received: from 10.194.205.88\r\n by atlas103.s...,pedidos@riooffsite.com.br|pedidos@riooffsite.c...,2022-04-01T00:56:38.000000+0000|2022-04-01T00:...,u10225017.ct.sendgrid.net|usaa.com|u10225017.c...


In [8]:
def count_from(row):
        return len(row["from"].split("|"))

events["from_count"] = events.apply(lambda row: count_from(row), axis=1)

events.head()

,index,id,date,attribute_count,Tag,header,from,feedback_time,host,from_count
0,19625,287833,2022-04-01,28,0,Delivered-To: UNDISCLOSEDFORPRIVACY@PRIVACYDOM...,earleenlpwoodman2593@gmail.com|earleenlpwoodma...,2022-03-31T23:21:12.000000+0000|2022-03-31T23:...,www1.nyc.gov|www1.nyc.gov,2
1,19626,287834,2022-04-01,79,0,Delivered-To: UNDISCLOSEDFORPRIVACY@PRIVACYDOM...,quickbooks@notification.intuit.com|quickbooks@...,2022-04-01T00:26:28.000000+0000|2022-04-01T00:...,links.notification.intuit.com|links.notificati...,2
2,19627,287835,2022-04-01,89,1,Received: from 10.194.205.88\r\n by atlas109.s...,fortune@msg.fortune.com|fortune@msg.fortune.com,2022-04-01T00:58:47.000000+0000|2022-04-01T00:...,fortune.com|fortune.com|fortune.com|fortune.co...,2
3,19628,287836,2022-04-01,159,1,Received: from 10.213.249.35\r\n by atlas106.s...,miltongrovesc.yahoo.com@send.mailchimpapp.com|...,2022-04-01T00:55:55.000000+0000|2022-04-01T00:...,mailchi.mp|miltongrovesc.us15.list-manage.com|...,2
4,19629,287837,2022-04-01,62,0,Received: from 10.194.205.88\r\n by atlas103.s...,pedidos@riooffsite.com.br|pedidos@riooffsite.c...,2022-04-01T00:56:38.000000+0000|2022-04-01T00:...,u10225017.ct.sendgrid.net|usaa.com|u10225017.c...,2


In [9]:
events["from_count"].unique()

array([2, 1, 3, 4], dtype=int64)

In [10]:
def count_host(row):
        return len(row["host"].split("|"))

events["host_count"] = events.apply(lambda row: count_host(row), axis=1)

events.head()

,index,id,date,attribute_count,Tag,header,from,feedback_time,host,from_count,host_count
0,19625,287833,2022-04-01,28,0,Delivered-To: UNDISCLOSEDFORPRIVACY@PRIVACYDOM...,earleenlpwoodman2593@gmail.com|earleenlpwoodma...,2022-03-31T23:21:12.000000+0000|2022-03-31T23:...,www1.nyc.gov|www1.nyc.gov,2,2
1,19626,287834,2022-04-01,79,0,Delivered-To: UNDISCLOSEDFORPRIVACY@PRIVACYDOM...,quickbooks@notification.intuit.com|quickbooks@...,2022-04-01T00:26:28.000000+0000|2022-04-01T00:...,links.notification.intuit.com|links.notificati...,2,12
2,19627,287835,2022-04-01,89,1,Received: from 10.194.205.88\r\n by atlas109.s...,fortune@msg.fortune.com|fortune@msg.fortune.com,2022-04-01T00:58:47.000000+0000|2022-04-01T00:...,fortune.com|fortune.com|fortune.com|fortune.co...,2,16
3,19628,287836,2022-04-01,159,1,Received: from 10.213.249.35\r\n by atlas106.s...,miltongrovesc.yahoo.com@send.mailchimpapp.com|...,2022-04-01T00:55:55.000000+0000|2022-04-01T00:...,mailchi.mp|miltongrovesc.us15.list-manage.com|...,2,30
4,19629,287837,2022-04-01,62,0,Received: from 10.194.205.88\r\n by atlas103.s...,pedidos@riooffsite.com.br|pedidos@riooffsite.c...,2022-04-01T00:56:38.000000+0000|2022-04-01T00:...,u10225017.ct.sendgrid.net|usaa.com|u10225017.c...,2,10


In [11]:
events["host_count"].unique()

array([  2,  12,  16,  30,  10,  24,  14,   4,   6, 206,   1,   8,  32,
        46,  96, 148,  20,  42,  22,  48,  40, 332,  72, 142,  70,  18,
         5,  34, 106, 144,  64,  50, 146,  26, 238,  28,  36,  44,  68,
       152,  56, 116, 202, 154, 143, 112,   3,  37,   9,  73,  23,   7,
        39,  11,  13,  27,  15,  17,  35,  19, 140,  33,  52,  38, 108,
        55,  88,  67,  91,  29,  53,  21,  25, 124,  45, 115, 132,  31,
        75,  63,  65,  43,  78,  66,  58, 120, 147,  54,  84, 135,  57,
       105,  51,  99, 165,  69,  81, 129, 138,  90,  98,  62, 190, 103,
        86,  60,  82, 104, 100, 102, 216,  94,  76,  71,  93, 114, 188,
       122, 118, 208, 126, 164,  80, 150,  74,  61,  41, 145,  47, 125,
       184, 186, 171, 153, 231, 210, 189, 240, 111, 200,  87, 201, 177,
       162, 141, 496, 212, 169, 285, 160,  92, 121, 159, 117, 192, 315,
       258, 195,  49,  77, 421,  59, 361, 127, 123, 128, 244, 276, 174,
       134, 232, 228, 178, 220, 130, 587, 349,  79, 209,  97, 15

In [12]:
import numpy as np

events["host_count_ln"] = events.apply(lambda row: np.log(row["host_count"]), axis=1)

events.head()

,index,id,date,attribute_count,Tag,header,from,feedback_time,host,from_count,host_count,host_count_ln
0,19625,287833,2022-04-01,28,0,Delivered-To: UNDISCLOSEDFORPRIVACY@PRIVACYDOM...,earleenlpwoodman2593@gmail.com|earleenlpwoodma...,2022-03-31T23:21:12.000000+0000|2022-03-31T23:...,www1.nyc.gov|www1.nyc.gov,2,2,0.693147
1,19626,287834,2022-04-01,79,0,Delivered-To: UNDISCLOSEDFORPRIVACY@PRIVACYDOM...,quickbooks@notification.intuit.com|quickbooks@...,2022-04-01T00:26:28.000000+0000|2022-04-01T00:...,links.notification.intuit.com|links.notificati...,2,12,2.484907
2,19627,287835,2022-04-01,89,1,Received: from 10.194.205.88\r\n by atlas109.s...,fortune@msg.fortune.com|fortune@msg.fortune.com,2022-04-01T00:58:47.000000+0000|2022-04-01T00:...,fortune.com|fortune.com|fortune.com|fortune.co...,2,16,2.772589
3,19628,287836,2022-04-01,159,1,Received: from 10.213.249.35\r\n by atlas106.s...,miltongrovesc.yahoo.com@send.mailchimpapp.com|...,2022-04-01T00:55:55.000000+0000|2022-04-01T00:...,mailchi.mp|miltongrovesc.us15.list-manage.com|...,2,30,3.401197
4,19629,287837,2022-04-01,62,0,Received: from 10.194.205.88\r\n by atlas103.s...,pedidos@riooffsite.com.br|pedidos@riooffsite.c...,2022-04-01T00:56:38.000000+0000|2022-04-01T00:...,u10225017.ct.sendgrid.net|usaa.com|u10225017.c...,2,10,2.302585


In [13]:
#use a regex to recover the subject of the mail from the header

import re

regex = re.compile(r'(?<=Subject: )(.*)(?=\n)')
events["subject"] = events.apply(lambda row: regex.search(row["header"]).group() if regex.search(row["header"]) is not None else "", axis=1)

events.head()

,index,id,date,attribute_count,Tag,header,from,feedback_time,host,from_count,host_count,host_count_ln,subject
0,19625,287833,2022-04-01,28,0,Delivered-To: UNDISCLOSEDFORPRIVACY@PRIVACYDOM...,earleenlpwoodman2593@gmail.com|earleenlpwoodma...,2022-03-31T23:21:12.000000+0000|2022-03-31T23:...,www1.nyc.gov|www1.nyc.gov,2,2,0.693147,Welcome to Your Survey-6668
1,19626,287834,2022-04-01,79,0,Delivered-To: UNDISCLOSEDFORPRIVACY@PRIVACYDOM...,quickbooks@notification.intuit.com|quickbooks@...,2022-04-01T00:26:28.000000+0000|2022-04-01T00:...,links.notification.intuit.com|links.notificati...,2,12,2.484907,Invoice 30148805 from Norton Billing
2,19627,287835,2022-04-01,89,1,Received: from 10.194.205.88\r\n by atlas109.s...,fortune@msg.fortune.com|fortune@msg.fortune.com,2022-04-01T00:58:47.000000+0000|2022-04-01T00:...,fortune.com|fortune.com|fortune.com|fortune.co...,2,16,2.772589,CBD - The Hottest Thing In 2022 - New Year New...
3,19628,287836,2022-04-01,159,1,Received: from 10.213.249.35\r\n by atlas106.s...,miltongrovesc.yahoo.com@send.mailchimpapp.com|...,2022-04-01T00:55:55.000000+0000|2022-04-01T00:...,mailchi.mp|miltongrovesc.us15.list-manage.com|...,2,30,3.401197,Happy Dietday!
4,19629,287837,2022-04-01,62,0,Received: from 10.194.205.88\r\n by atlas103.s...,pedidos@riooffsite.com.br|pedidos@riooffsite.c...,2022-04-01T00:56:38.000000+0000|2022-04-01T00:...,u10225017.ct.sendgrid.net|usaa.com|u10225017.c...,2,10,2.302585,Verify mobile number on your profile


In [14]:
events.to_csv("events_augmented.csv")
